In [1]:
import os
import torch
import torch.backends.cudnn as cudnn
from PIL import Image
import numpy as np
import cv2

from split import train
from modules.split_modules import SplitModel

In [2]:
train_img_dir = '/Users/longhoang/Developer/table-reg/data/scitsr-split-train/train/img'
train_json_label = '/Users/longhoang/Developer/table-reg/data/scitsr-split-train/train/label/split-label.json'
val_img_dir = '/Users/longhoang/Developer/table-reg/data/scitsr-split-train/val/img'
val_json_label = '/Users/longhoang/Developer/table-reg/data/scitsr-split-train/val/label/split-label.json'
save_dir = '/Users/longhoang/Developer/table-reg/data/scitsr-split-train/result'

In [3]:
# !python ../fireae-split-merge/split/train.py --img_dir {train_img_dir} --json_dir {train_json_label} --saved_dir {save_dir} --val_img_dir {val_img_dir} --val_json {val_json_label}

In [4]:
class Args:
    def __init__(self, train_img_dir, train_json_label, val_img_dir, val_json_label, save_dir,
                 batch_size=1, epochs=50, gpu=True, gpu_list='0', lr=0.00075,
                 featureW=8, scale=0.5):
        self.img_dir = train_img_dir
        self.json_dir = train_json_label
        self.saved_dir = save_dir
        self.val_img_dir = val_img_dir
        self.val_json = val_json_label
        self.batch_size = batch_size
        self.epochs = epochs
        self.gpu = gpu
        self.gpu_list = gpu_list
        self.lr = lr
        self.featureW = featureW
        self.scale = scale

In [5]:
opt = Args(train_img_dir, train_json_label, val_img_dir, val_json_label, save_dir, gpu=False)
net = SplitModel(3)
if opt.gpu:
    cudnn.benchmark = True
    cudnn.deterministic = True
    os.environ['CUDA_VISIBLE_DEVICES'] = opt.gpu_list
    net = torch.nn.DataParallel(net).cuda()

if not os.path.exists(opt.saved_dir):
    os.mkdir(opt.saved_dir)

train.train(opt, net)

Data loaded!
epoch:1
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


[W NNPACK.cpp:64] Could not initialize NNPACK! Reason: Unsupported hardware.


KeyboardInterrupt: 

In [ ]:
img_path = os.path.join(opt.img_dir, os.listdir(opt.img_dir)[0])
img = cv2.imread(img_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = np.transpose(img, (2,0,1))
img.shape

(3, 233, 478)